In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [15]:
# Assuming preprocessed data
# X is the input matrix with shape (num_samples, sequence_length)
# y are your labels (0 or 1 for negative or positive sentiment)

# Mock dataset for demonstration
class TextDataset(Dataset):
    def __init__(self, vocabulary_size, sequence_length, num_samples):
        self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
        self.labels = torch.randint(0, 2, (num_samples,))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Define the Q-network model
class DQN(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
        super(DQN, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        q_values = self.fc(lstm_out[:, -1])
        return q_values

def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Using CrossEntropyLoss which combines LogSoftmax and NLLLoss
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')


In [22]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class TextDataset(Dataset):
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i+1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab

    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Assuming `texts` and `labels` are correctly defined lists above
sequence_length = 10  # Define the desired sequence length

# Correct initialization of datasets
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Proceed to create DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity


Validation Encoded text: tensor([[ 2, 73, 16, 17,  0,  0,  0,  0,  0,  0],
        [ 1, 53,  0,  0,  0,  0,  0,  0,  0,  0],
        [52,  3, 26, 63, 30,  0,  0,  0,  0,  0],
        [74,  9,  0,  0,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 0, 0])
Validation Encoded text: tensor([[21, 10,  4,  0,  0,  0,  0,  0,  0,  0],
        [22, 28, 16,  8,  0,  0,  0,  0,  0,  0],
        [22, 12, 30,  2, 19, 20,  0,  0,  0,  0],
        [11,  6, 13, 29,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[19, 21,  3, 18, 28,  0,  0,  0,  0,  0],
        [ 1, 14, 25, 16,  1, 23, 12,  0,  0,  0],
        [26, 29,  7,  0,  0,  0,  0,  0,  0,  0],
        [22, 11,  2,  0,  0,  0,  0,  0,  0,  0]])
Test Label: tensor([0, 0, 0, 0])


In [23]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Dataset and DataLoader
train_dataset = TextDataset(vocabulary_size, sequence_length, 1000)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)

# Add a validation dataset
validation_dataset = TextDataset(vocabulary_size, sequence_length, 200)  # Mocked
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

# Add a test dataset
test_dataset = TextDataset(vocabulary_size, sequence_length, 200)  # Mocked
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

TypeError: 'int' object is not iterable

In [ ]:
# Modify the training loop to include validation
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)


Train Epoch: 1 [0/1000 (0%)]	Loss: 0.007577
Train Epoch: 1 [640/1000 (62%)]	Loss: 0.008206

Validation set: Average loss: 0.0386, Accuracy: 98/200 (49%)

Train Epoch: 2 [0/1000 (0%)]	Loss: 0.005453
Train Epoch: 2 [640/1000 (62%)]	Loss: 0.002371

Validation set: Average loss: 0.0434, Accuracy: 97/200 (48%)

Train Epoch: 3 [0/1000 (0%)]	Loss: 0.003870
Train Epoch: 3 [640/1000 (62%)]	Loss: 0.001445

Validation set: Average loss: 0.0466, Accuracy: 92/200 (46%)

Train Epoch: 4 [0/1000 (0%)]	Loss: 0.002094
Train Epoch: 4 [640/1000 (62%)]	Loss: 0.001928

Validation set: Average loss: 0.0495, Accuracy: 95/200 (48%)

Train Epoch: 5 [0/1000 (0%)]	Loss: 0.001457
Train Epoch: 5 [640/1000 (62%)]	Loss: 0.001234

Validation set: Average loss: 0.0539, Accuracy: 89/200 (44%)

Train Epoch: 6 [0/1000 (0%)]	Loss: 0.001832
Train Epoch: 6 [640/1000 (62%)]	Loss: 0.001397

Validation set: Average loss: 0.0489, Accuracy: 95/200 (48%)

Train Epoch: 7 [0/1000 (0%)]	Loss: 0.000856
Train Epoch: 7 [640/1000 (62%)]	